In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


train = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Pulpit\MyProjects\titanic\train.csv')
test = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Pulpit\MyProjects\titanic\test.csv')
gender_sub = pd.read_csv(r'C:\Users\Lenovo\OneDrive\Pulpit\MyProjects\titanic\gender_submission.csv')
#train_y = train['Survived']
#test_y = test['Survived']

train_y = train['Survived']
test_y=gender_sub['Survived']



In [36]:
train = train.drop('Survived', axis=1)

In [37]:
data=[train,test]
for dataset in data:
    dataset['Sex']=dataset['Sex'].map({'male':0, 'female':1})
    dataset['Prefix'] = dataset['Name'].str.split(expand=True)[1]

In [38]:
train = train.drop('PassengerId', axis=1)
test = test.drop('PassengerId', axis=1)

In [39]:
prefix = train['Prefix'].value_counts()
prefix
top_prefixes = prefix[prefix>len(train)/25]
top_prefixes

Mr.        502
Miss.      179
Mrs.       121
Master.     40
Name: Prefix, dtype: int64

In [40]:
train['Prefix'] = train['Prefix'].apply(lambda x: x if x in top_prefixes else 'Other')
test['Prefix'] = test['Prefix'].apply(lambda x: x if x in top_prefixes else 'Other')

In [41]:
train.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
Prefix        0
dtype: int64

In [42]:
test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
Prefix        0
dtype: int64

In [43]:
#Dropping the columns that have more than 70% of data missing
cols_null = train.columns[train.isnull().sum()>len(train)*0.7]

In [44]:
train = train.drop(cols_null, axis=1)
test = test.drop(cols_null, axis=1)


In [45]:
#we are substituting one missing value with the mean value for Fare
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())


In [46]:
train.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

In [47]:
#creating more generalized categories for fare
train['Fare_cat'] = pd.qcut(train.Fare, q=4, labels=[1,2,3,4]) #categories based on price of ticket
test['Fare_cat'] = pd.qcut(test.Fare, q=4, labels=[1,2,3,4])
train['Fare_cat'] = train['Fare_cat'].astype('int64')
test['Fare_cat'] = test['Fare_cat'].astype('int64')

In [48]:
#now that we have fare categories, we can drop 'Fare' column
train=train.drop('Fare', axis=1)
test = test.drop('Fare', axis=1)

In [49]:
#making combined category for family on board
data=[train,test]
for dataset in data:
    dataset['Fam'] = dataset['SibSp']+dataset['Parch']
    

In [50]:
train = train.drop(['SibSp', 'Parch'], axis=1)
test = test.drop(['SibSp', 'Parch'], axis=1)

In [51]:
train.isnull().sum()

Pclass        0
Name          0
Sex           0
Age         177
Ticket        0
Embarked      2
Prefix        0
Fare_cat      0
Fam           0
dtype: int64

In [52]:
train.dtypes

Pclass        int64
Name         object
Sex           int64
Age         float64
Ticket       object
Embarked     object
Prefix       object
Fare_cat      int64
Fam           int64
dtype: object

In [53]:
#as we got prefix we can get rid of 'Name' category
#we can also drop information about the ticket since we decided to drop 'Cabin' column due to amount of lost data
#the rest of object categories can be transformed by one hot encoding because they don't have too much unique values

train = train.drop(['Ticket', 'Name'], axis=1)
test = test.drop(['Ticket', 'Name'], axis=1)
obj_cats = train.columns[train.dtypes=='object']


In [54]:
obj_cats

Index(['Embarked', 'Prefix'], dtype='object')

In [55]:
num_cats = train.columns[train.dtypes!='object']

In [56]:
num_cats

Index(['Pclass', 'Sex', 'Age', 'Fare_cat', 'Fam'], dtype='object')

In [57]:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

num_cols = train[num_cats]
cat_cols = train[obj_cats]
pipeline_cat = Pipeline([('one_hot', OneHotEncoder(handle_unknown='ignore'))])
pipeline_num = Pipeline([('imputer', SimpleImputer(strategy='mean'))])
cat_transformed = pipeline_cat.fit_transform(cat_cols)

#cat_transformed = pd.DataFrame(cat_transformed, columns = cat_cols.columns, index= cat_cols.index)
preprocessor = ColumnTransformer(
    transformers = [('num', pipeline_num, num_cols), ('cat', pipeline_cat, cat_cols)])
preprocessor
                                 

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                      Pclass  Sex   Age  Fare_cat  Fam
0         3    0  22.0         1    1
1         1    1  38.0         4    1
2         3    1  26.0         2    0
3         1    1  35.0         4    1
4         3    0  35.0         2    0
..      ...  ...   ...       ...  ...
886       2    0  27.0         2    0
887       1    1  19.0         3    0
888       3    1   NaN         3    3
889       1    0  26.0         3    0
890       3    0  32.0         1    0

[891 rows x 5 columns]),
                                ('cat',
                                 Pipeline(steps=[('one_hot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                     Embarked Prefix
0          S    Mr.
1          C   Mrs.
2          S  Miss.
3          S   Mrs.
4          S    Mr.
..       ...    ...
886        S  Other
887        S  Miss.
888        S  Miss.
889        C    Mr.
890        Q    Mr.

[891 rows x 2 columns])])

In [58]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=42)

pipeline_model = Pipeline([('preprocessor', preprocessor), ('model', model)])
pipeline_model.fit(train, train_y)

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed